In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=['Unnamed: 0','fechapostulacion'])

In [3]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         uint32
idpostulante    object
dtypes: object(1), uint32(1)
memory usage: 75.6+ MB


In [4]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
#postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'],axis=1)
#postulantes_genero_edad

In [6]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['nombre_numerico'])

In [7]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [8]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504400 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante       504400 non-null object
fechanacimiento    504400 non-null object
sexo               504400 non-null category
sexo_numerico      504400 non-null int64
nombre             447906 non-null category
estado             447906 non-null category
dtypes: category(3), int64(1), object(2)
memory usage: 16.8+ MB


In [9]:
postulantes_merge.head()

,idpostulante,fechanacimiento,sexo,sexo_numerico,nombre,estado
0,zvxzzL6,1989-08-31,MASC,2,Secundario,Graduado
1,zvxzvD5,1994-10-10,FEM,1,Terciario/Técnico,En Curso
2,zvxzrWm,1981-11-30,MASC,2,Universitario,Graduado
3,zvxzmED,1999-04-09,FEM,1,Secundario,Graduado
4,zvxzld5,1997-12-17,MASC,2,NaN,NaN


In [10]:
#Voy a agregar una columna con la edad ya calculada(estimando por 2017-año nacimiento) para cada registro
postulantes_merge["año"] = postulantes_merge[postulantes_merge["fechanacimiento"] != 'nan']\
        ["fechanacimiento"].apply(lambda x: x[0:4])
postulantes_merge["edad"] = postulantes_merge[postulantes_merge["año"].notna()] \
        ["año"].apply(lambda x: 2017 - int(x))

In [11]:
#Elimino columna fechanacimiento y año que ya no me sirven
postulantes_merge = postulantes_merge.drop(columns=['fechanacimiento','año'])

In [12]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504400 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     504400 non-null object
sexo             504400 non-null category
sexo_numerico    504400 non-null int64
nombre           447906 non-null category
estado           447906 non-null category
edad             478315 non-null float64
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 16.8+ MB


In [13]:
#Elimino NaNs
postulantes_merge = postulantes_merge[postulantes_merge['nombre'].notnull() &
                                     postulantes_merge['estado'].notnull() &
                                     postulantes_merge['edad'].notnull()]

In [14]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438702 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     438702 non-null object
sexo             438702 non-null category
sexo_numerico    438702 non-null int64
nombre           438702 non-null category
estado           438702 non-null category
edad             438702 non-null float64
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 14.6+ MB


In [15]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
postulantes_merge["estado_code"] = lb_make1.fit_transform(postulantes_merge["estado"])

lb_make2 = LabelEncoder()
postulantes_merge["sexo_code"] = lb_make2.fit_transform(postulantes_merge["sexo"])

lb_make3 = LabelEncoder()
postulantes_merge["nombre_code"] = lb_make3.fit_transform(postulantes_merge["nombre"])

postulantes_merge.drop(columns=['sexo','nombre','estado'],inplace=True)
postulantes_merge[['sexo_code','nombre_code','estado_code','edad']] = postulantes_merge[['sexo_code','nombre_code','estado_code','edad']].apply(pd.to_numeric,downcast='unsigned')

In [16]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438702 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     438702 non-null object
sexo_numerico    438702 non-null int64
edad             438702 non-null uint16
estado_code      438702 non-null uint8
sexo_code        438702 non-null uint8
nombre_code      438702 non-null uint8
dtypes: int64(1), object(1), uint16(1), uint8(3)
memory usage: 12.1+ MB


In [17]:
postulantes_merge.to_csv('/home/fabrizio/env/NaventDatosTP/Data/POSTULANTES_FINAL.csv')

# Agrego la informacion a postulaciones

In [18]:
merge_postulaciones = pd.merge(postulaciones,postulantes_merge,how='left',on='idpostulante')

In [19]:
#Creo columna con un 1 para indicar postulacion
merge_postulaciones['se_postulo'] = 1
merge_postulaciones['se_postulo'] = merge_postulaciones[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [20]:
merge_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 8 columns):
idaviso          uint32
idpostulante     object
sexo_numerico    float64
edad             float64
estado_code      float64
sexo_code        float64
nombre_code      float64
se_postulo       uint8
dtypes: float64(5), object(1), uint32(1), uint8(1)
memory usage: 384.2+ MB


In [21]:
merge_postulaciones[merge_postulaciones['edad'].isnull()]

,idaviso,idpostulante,sexo_numerico,edad,estado_code,sexo_code,nombre_code,se_postulo
700,1112281337,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
701,1112334640,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
702,1112289378,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
703,1112296073,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
704,1112310231,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
705,1112195462,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
706,1112288872,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
707,1112277388,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
708,1112334643,NM1J0z,NaN,NaN,NaN,NaN,NaN,1
709,1112153209,NM1J0z,NaN,NaN,NaN,NaN,NaN,1


In [22]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
merge_sin_nan = merge_postulaciones[merge_postulaciones['nombre_code'].notnull() &
                                    merge_postulaciones['estado_code'].notnull() & 
                                    merge_postulaciones['edad'].notnull()]
merge_sin_nan = merge_sin_nan[merge_sin_nan['nombre_code'].notnull() &
                                    merge_sin_nan['estado_code'].notnull()]
merge_sin_nan[['edad','estado_code','sexo_code','nombre_code']] = merge_sin_nan[['edad','estado_code','sexo_code','nombre_code']].apply(pd.to_numeric,downcast='unsigned')
merge_sin_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6330266 entries, 0 to 6603749
Data columns (total 8 columns):
idaviso          uint32
idpostulante     object
sexo_numerico    float64
edad             uint16
estado_code      uint8
sexo_code        uint8
nombre_code      uint8
se_postulo       uint8
dtypes: float64(1), object(1), uint16(1), uint32(1), uint8(4)
memory usage: 205.3+ MB


# Agrego datos de vistas que serian los postulantes que ven algo y no se postulan

In [23]:
dtypes = pd.Series({'idAviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

vistas = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Vistas/Vistas-merge.csv',dtype=column_types,parse_dates=['timestamp'],
                     infer_datetime_format=True)
vistas = vistas.drop(columns=['Unnamed: 0'])
#Considero unica vista de aviso por postulante (no me importa ni horario ni cantidad solo que lo vea)
vistas.drop(columns=['timestamp'],inplace=True)
vistas.drop_duplicates(subset=['idAviso','idpostulante'],inplace=True)
vistas['se_postulo'] = 0
vistas['se_postulo'] = vistas[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [24]:
vistas.rename(index=str,columns={'idAviso':'idaviso'},inplace=True)
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 172.2+ MB


# Agrego datos de avisos y lo mergeo con las vistas y las postulaciones

In [25]:
avisos = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/optimized_avisos_detalle.csv')
avisos.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [26]:
avisos.isnull().sum()

idaviso                      0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [27]:
#Hago un merge de avisos y vistas
vistas_y_avisos = pd.merge(vistas,avisos,how='inner',on='idaviso')

In [28]:
vistas_y_avisos[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']] = vistas_y_avisos[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']].apply(pd.to_numeric,downcast='unsigned')
vistas_y_avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5542237 entries, 0 to 5542236
Data columns (total 8 columns):
idaviso                      uint32
idpostulante                 object
se_postulo                   uint8
nombre_area_code             uint8
denominacion_empresa_code    uint16
nivel_laboral_code           uint8
tipo_de_trabajo_code         uint8
nombre_zona_code             uint8
dtypes: object(1), uint16(1), uint32(1), uint8(5)
memory usage: 142.7+ MB


In [ ]:
merge_sin_nan = pd.merge(merge_sin_nan,avisos,how='inner',on='idaviso')

# Merge de todos

In [29]:
#Puede que haya vistas que enrealidad el postulante luego se postulo asi que verifico con el dataset merge
outer = pd.merge(merge_sin_nan,vistas_y_avisos,how='outer',on=['idaviso','idpostulante'])

In [30]:
outer.se_postulo_x.value_counts()

1.0    6330266
Name: se_postulo_x, dtype: int64

In [31]:
outer.se_postulo_y.value_counts()

0.0    5542237
Name: se_postulo_y, dtype: int64

In [32]:
outer['se_postulo_x'].fillna(0, inplace=True)
outer['se_postulo_y'].fillna(0, inplace=True)

In [33]:
outer.se_postulo_x.value_counts()

1.0    6330266
0.0    3864754
Name: se_postulo_x, dtype: int64

In [34]:
outer['se_postulo'] = outer['se_postulo_x'] + outer['se_postulo_y']

In [35]:
outer.se_postulo.value_counts()

1.0    6330266
0.0    3864754
Name: se_postulo, dtype: int64

In [36]:
outer.isnull().sum()

idaviso                            0
idpostulante                       0
sexo_numerico                3864754
edad                         3864754
estado_code                  3864754
sexo_code                    3864754
nombre_code                  3864754
se_postulo_x                       0
se_postulo_y                       0
nombre_area_code             4652783
denominacion_empresa_code    4652783
nivel_laboral_code           4652783
tipo_de_trabajo_code         4652783
nombre_zona_code             4652783
se_postulo                         0
dtype: int64

In [37]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
outer1 = outer[outer['nombre_code'].notnull() &
            outer['estado_code'].notnull() & 
            outer['edad'].notnull()]

In [38]:
outer2 = outer[outer['nombre_area_code'].notnull() &
             outer['denominacion_empresa_code'].notnull() &
             outer['nivel_laboral_code'].notnull() &
             outer['tipo_de_trabajo_code'].notnull() &
             outer['nombre_zona_code'].notnull()]

In [41]:
outer = outer[outer['nombre_code'].notnull() &
            outer['estado_code'].notnull() & 
            outer['edad'].notnull() &]

outer1.info()


In [ ]:
outer2 = outer[outer['nombre_area_code'].notnull() &
             outer['denominacion_empresa_code'].notnull() &
             outer['nivel_laboral_code'].notnull() &
             outer['tipo_de_trabajo_code'].notnull() &
             outer['nombre_zona_code'].notnull()] 

In [40]:
outer2.isnull().sum()

idaviso                            0
idpostulante                       0
sexo_numerico                3864754
edad                         3864754
estado_code                  3864754
sexo_code                    3864754
nombre_code                  3864754
se_postulo_x                       0
se_postulo_y                       0
nombre_area_code                   0
denominacion_empresa_code          0
nivel_laboral_code                 0
tipo_de_trabajo_code               0
nombre_zona_code                   0
se_postulo                         0
dtype: int64

In [43]:
#Dejo solo los que no aparecen ya en postulaciones
trainingSet = outer

In [ ]:
trainingSet

In [ ]:
trainingSet.se_postulo_y.value_counts()

In [ ]:
trainingSet['se_postulo'] = trainingSet['se_postulo_x'] + trainingSet['se_postulo_y']

In [ ]:
trainingSet.se_postulo.value_counts()

In [44]:
trainingSet.drop('se_postulo_x',axis=1,inplace=True)
trainingSet.drop('se_postulo_y',axis=1,inplace=True)
#trainingSet.rename(index=str,columns={'se_postulo_y':'se_postulo'},inplace=True)

In [45]:
trainingSet.se_postulo.value_counts()

1.0    1677483
Name: se_postulo, dtype: int64

In [46]:
postMasVistas = pd.concat([trainingSet,merge_sin_nan])

In [47]:
postMasVistas.isnull().sum()

denominacion_empresa_code    6330266
edad                               0
estado_code                        0
idaviso                            0
idpostulante                       0
nivel_laboral_code           6330266
nombre_area_code             6330266
nombre_code                        0
nombre_zona_code             6330266
se_postulo                         0
sexo_code                          0
sexo_numerico                      0
tipo_de_trabajo_code         6330266
dtype: int64

In [48]:
postMasVistas = postMasVistas[postMasVistas['denominacion_empresa_code'].notnull() &
                             postMasVistas['nivel_laboral_code'].notnull() &
                             postMasVistas['nombre_area_code'].notnull() &
                             postMasVistas['nombre_zona_code'].notnull() &
                             postMasVistas['tipo_de_trabajo_code'].notnull()]

In [49]:
postMasVistas.isnull().sum()

denominacion_empresa_code    0
edad                         0
estado_code                  0
idaviso                      0
idpostulante                 0
nivel_laboral_code           0
nombre_area_code             0
nombre_code                  0
nombre_zona_code             0
se_postulo                   0
sexo_code                    0
sexo_numerico                0
tipo_de_trabajo_code         0
dtype: int64

In [50]:
postMasVistas.se_postulo.value_counts()

1.0    1677483
Name: se_postulo, dtype: int64

In [51]:
#postMasVistas.to_csv('/home/fabrizio/env/NaventDatosTP/Data/postulaciones_mas_vistas_mas_avisos.csv')

In [56]:
postMasVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1677483 entries, 23 to 6330265
Data columns (total 13 columns):
denominacion_empresa_code    1677483 non-null float64
edad                         1677483 non-null float64
estado_code                  1677483 non-null float64
idaviso                      1677483 non-null uint64
idpostulante                 1677483 non-null object
nivel_laboral_code           1677483 non-null float64
nombre_area_code             1677483 non-null float64
nombre_code                  1677483 non-null float64
nombre_zona_code             1677483 non-null float64
se_postulo                   1677483 non-null float64
sexo_code                    1677483 non-null float64
sexo_numerico                1677483 non-null float64
tipo_de_trabajo_code         1677483 non-null float64
dtypes: float64(11), object(1), uint64(1)
memory usage: 179.2+ MB


In [57]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistas.drop('idpostulante',axis=1)

In [58]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistasCopiaSinPostulanteParaCorr.drop('idaviso',axis=1)

In [59]:
postMasVistasCopiaSinPostulanteParaCorr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1677483 entries, 23 to 6330265
Data columns (total 11 columns):
denominacion_empresa_code    1677483 non-null float64
edad                         1677483 non-null float64
estado_code                  1677483 non-null float64
nivel_laboral_code           1677483 non-null float64
nombre_area_code             1677483 non-null float64
nombre_code                  1677483 non-null float64
nombre_zona_code             1677483 non-null float64
se_postulo                   1677483 non-null float64
sexo_code                    1677483 non-null float64
sexo_numerico                1677483 non-null float64
tipo_de_trabajo_code         1677483 non-null float64
dtypes: float64(11)
memory usage: 153.6 MB


In [61]:
postMasVistasCopiaSinPostulanteParaCorr.corr()

,denominacion_empresa_code,edad,estado_code,nivel_laboral_code,nombre_area_code,nombre_code,nombre_zona_code,se_postulo,sexo_code,sexo_numerico,tipo_de_trabajo_code
denominacion_empresa_code,1.000000,0.011088,0.000296,-0.029774,-0.026993,0.001183,-0.073862,NaN,0.005257,0.005173,-0.014021
edad,0.011088,1.000000,0.059053,-0.010369,-0.029921,-0.061595,0.001050,NaN,0.087349,0.082083,-0.063310
estado_code,0.000296,0.059053,1.000000,0.014081,0.039937,-0.351224,0.009062,NaN,0.056156,0.053730,-0.039240
nivel_laboral_code,-0.029774,-0.010369,0.014081,1.000000,0.057038,-0.022599,0.080106,NaN,-0.031285,-0.031078,-0.065426
nombre_area_code,-0.026993,-0.029921,0.039937,0.057038,1.000000,-0.080921,0.032350,NaN,0.000442,0.000381,0.021906
nombre_code,0.001183,-0.061595,-0.351224,-0.022599,-0.080921,1.000000,-0.012481,NaN,-0.083417,-0.080161,0.023432
nombre_zona_code,-0.073862,0.001050,0.009062,0.080106,0.032350,-0.012481,1.000000,NaN,0.001660,0.001508,-0.008008
se_postulo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo_code,0.005257,0.087349,0.056156,-0.031285,0.000442,-0.083417,0.001660,NaN,1.000000,0.964310,-0.071786
sexo_numerico,0.005173,0.082083,0.053730,-0.031078,0.000381,-0.080161,0.001508,NaN,0.964310,1.000000,-0.071482


In [55]:
postMasVistasCopiaSinPostulanteParaCorr.se_postulo.value_counts()

1.0    1677483
Name: se_postulo, dtype: int64

In [ ]:
#Dropeo las columnas que use para codificar
postulantes_merge = postulantes_merge.drop(columns=['sexo','nombre','estado'])
postulantes_merge.info()

In [ ]:
postulantes_merge_int = postulantes_merge.select_dtypes(include=['int','float'])
converted_int = postulantes_merge_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(postulantes_merge_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([postulantes_merge_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
optimized_postulantes_merge = postulantes_merge.copy()

optimized_postulantes_merge[converted_int.columns] = converted_int

print(mem_usage(postulantes_merge))
print(mem_usage(optimized_postulantes_merge))

In [ ]:
optimized_postulantes_merge.info()

In [ ]:
optimized_postulantes_merge.to_csv('Data/optimized_postulantes_merge.csv')

In [ ]:
intereses_vistas = pd.read_csv('Data/unique_postulantes_con_intereses_vistas.csv')

In [ ]:
intereses_vistas.head()